# Random Forest Classifier x Toxic Content Detection
Il presente Notebook mostra l'addestramento ed il testing di un Classificatore basato su Random Forest per il task di Toxic Content Detection.

I dati sono stati processati come segue:
1. Pulizia del testo (si veda, 'dataset_preprocessing.py')
2. Lemmatizzazione con NLTK
3. Vettorizzazione con TF-IDF

In [2]:
import pandas as pd
import pickle
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
from sklearn.metrics import accuracy_score

# Addestramento del Sistema
Il Sistema è ovviamente riaddestrabile a piacere. Si consiglia, tuttavia, dato il tempo necessario per riaddestrare il classificatore, di utilizzare il file pickle 'rf_classifier' per eseguire subito gli esperimenti.

## Caricamento del Training Set

In [3]:
training_set = pd.read_csv("./../../datasets/training_set.csv")
training_set_lem = pd.read_csv("./../../datasets/training_set_lemmatized.csv")

# Osservazione: il Training Set è stato già ripulito
training_set

,comment_text,toxic
0,cocksucker before you piss around on my work,1
1,hey what is it talk what is it an exclusive gr...,1
2,bye dont look come or think of comming back to...,1
3,you are gay or antisemmitian archangel white t...,1
4,fuck your filthy mother in the ass dry,1
...,...,...
30572,chris i dont know who you are talking to but i...,0
30573,operation condor is also named a dirty war can...,0
30574,there is no evidence that this block has anyth...,0
30575,thanks hey utkarshraj thanks for the kindness ...,0


Sia l'addestramento che il testing saranno eseguiti sia sul Dataset "non-lemmatizzato" che sul Dataset "lemmatizzato". Osserviamo immediatamente che lo spazio delle feature del Dataset "lemmatizzato" è inferiore (49188 $<$ 56091) rispetto a quello del Dataset "non-lemmatizzato". Ciò ha impatto sia sul tempo necessario per addestrare il classificatore sia sull'accuracy del modello, come verrà mostrato in seguito.

In [4]:
# Vettorizzazione con TF-IDF
vectorizer = TfidfVectorizer()
vectorizer_lem = TfidfVectorizer()

X_train = vectorizer.fit_transform(training_set['comment_text'])
y_train = training_set['toxic']

X_train_lem = vectorizer_lem.fit_transform(training_set_lem['comment_text'])
y_train_lem = training_set_lem['toxic']

print("X_train.shape: " + str(X_train.shape))
print("y_train.shape: " + str(y_train.shape))

print("X_train_lem.shape: " + str(X_train_lem.shape))
print("y_train_lem.shape: " + str(y_train_lem.shape))

X_train.shape: (30577, 56091)
y_train.shape: (30577,)
X_train_lem.shape: (30577, 49188)
y_train_lem.shape: (30577,)


## Addestramento del Modello

In [5]:
import pickle
n_estimators = 100
model_filename = 'rf_classifier_{}.pkl'.format(n_estimators)
model_lem_filename = 'rf_classifier_lem_{}.pkl'.format(n_estimators)
cl, cl_lem = None, None

Esegui la seguente sottosezione per riaddestrare il Classificatore da capo. Il modello ottenuto verrà persistito nel file 'rf_classifier_{n_estimators}.pkl'

In [12]:
from sklearn.ensemble import RandomForestClassifier
cl = RandomForestClassifier(n_estimators=n_estimators)
cl_lem = RandomForestClassifier(n_estimators=n_estimators)

In [13]:
# Addestramento sul Dataset non-lemmatizzato

print("Estimators: " + str(n_estimators))
print("Training started on not-Lemmatized Dataset...")
start = datetime.now()
cl.fit(X=X_train, y=y_train)
end = datetime.now()
print("Training completed! Required time: " + str(end-start))

with open(model_filename, 'wb') as f:
    pickle.dump(cl, f)

Estimators: 100
Training started on not-Lemmatized Dataset...
Training completed! Required time: 0:00:32.082415


In [14]:
# Addestramento sul Dataset lemmatizzato

print("Estimators: " + str(n_estimators))
print("Training started on Lemmatized Dataset...")
start = datetime.now()
cl_lem.fit(X=X_train_lem, y=y_train_lem)
end = datetime.now()
print("Training completed! Required time: " + str(end-start))

with open(model_lem_filename, 'wb') as f:
    pickle.dump(cl_lem, f)

Estimators: 100
Training started on Lemmatized Dataset...
Training completed! Required time: 0:00:27.608418


Esegui la seguente sottosezione per utilizzare il Classificatore già addestrato.

In [11]:
with open(model_filename, 'rb') as f:
    cl = pickle.load(f)

ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]

In [9]:
with open(model_lem_filename, 'rb') as f:
    cl_lem = pickle.load(f)

ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]

# Testing del Sistema

In [15]:
test_set = pd.read_csv("./../../datasets/test_set.csv")
test_set_lem = pd.read_csv("./../../datasets/test_set_lemmatized.csv")

test_set.dropna(inplace=True)
test_set_lem.dropna(inplace=True)

In [16]:
test_set = test_set[test_set['toxic'] != -1]
other_set = test_set[test_set['toxic'] == -1]

test_set_lem = test_set_lem[test_set_lem['toxic'] != -1]
other_set_lem = test_set_lem[test_set_lem['toxic'] == -1]

In [17]:
X_test = vectorizer.transform(test_set['comment_text'])
y_test = test_set['toxic']

print("X_test.shape: " + str(X_test.shape))
print("y_test.shape: " + str(y_test.shape))

X_test.shape: (63842, 56091)
y_test.shape: (63842,)


In [18]:
X_test_lem = vectorizer_lem.transform(test_set_lem['comment_text'])
y_test_lem = test_set_lem['toxic']

print("X_test_lem.shape: " + str(X_test_lem.shape))
print("y_test_lem.shape: " + str(y_test_lem.shape))

X_test_lem.shape: (63842, 49188)
y_test_lem.shape: (63842,)


Predizioni sul Test Set non-Lemmatizzato

In [19]:
y_pred = cl.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8220920397230663


Predizioni sul Test Set Lemmatizzato

In [20]:
y_pred_lem = cl_lem.predict(X_test_lem)
print(accuracy_score(y_test_lem, y_pred_lem))

0.8227185865104477


### Funzioni di Supporto
Da utilizzare qualora si volesse effettuare una predizione su una frase "inedita".

In [21]:
# Pulizia della Frase
def clean_phrases(phrases):
    new_phrases = list()
    for phrase in phrases:
        # Rimozione di "\r" e "\n"
        phrase = re.sub(r'[\r\n]+', '', phrase)
        # Rimozione di sequenze di ":" (esempio, "::::")
        phrase = re.sub(r'::+', '', phrase)
        # Rimozione di sequenze di "=" (esempio, "====")
        phrase = re.sub(r'==+', '', phrase)
        # Rimozione di sequenze di "*" (esempio, "**")
        phrase = re.sub(r'\*\*+', '', phrase)
        # Rimozione di sequenze numeriche in formato di indirizzi IP (esempio, "192.168.1.1")
        phrase = re.sub(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', '', phrase)
        # Rimozione di contenuto compreso tra Parentesi Quadre (esempio, "[contentContent]")
        phrase = re.sub(r'\[[^\[\]]+\]', '', phrase)
        # Rimozione di Apici, sia singoli che doppi
        phrase = re.sub(r"['\"]", "", phrase)

        ## La rimozione di particolari caratteri o sequenze di caratteri può portare alla fusione di due token diversi

        # Splitting di token in cui compare un segno di interpuzione forte ("?", "!" e ".") seguito da una lettera maiuscola
        phrase = re.sub(r'([?!\.])([A-Z]\w*)', r'\1 \2', phrase)
        # Splitting di parole fuse (esempio, "parolaParola" diventa "parola Parola")
        phrase = re.sub(r'([a-z])([A-Z])', r'\1 \2', phrase)

        tokens = word_tokenize(phrase)
        lowercase_tokens = [token.lower() for token in tokens if token.isalpha()]
        new_phrases.append(' '.join(lowercase_tokens))
    
    return new_phrases

In [22]:
# Lemmatizzazione della Frase
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatized_text = []
    for token, tag in tagged_tokens:
        pos = get_wordnet_pos(tag)
        lemmatized_token = lemmatizer.lemmatize(token, pos=pos)
        lemmatized_text.append(lemmatized_token)
    return ' '.join(lemmatized_text)

Esempio:

In [23]:
text = "Today I went to the office at 9 o'clock and there were a lot of people."

In [30]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alessandropesare/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
phrases = clean_phrases([text])

string = phrases[0]
lemmatized_string = lemmatize_text(phrases[0])

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/Users/alessandropesare/nltk_data'
    - '/Users/alessandropesare/anaconda3/envs/DL_Project/nltk_data'
    - '/Users/alessandropesare/anaconda3/envs/DL_Project/share/nltk_data'
    - '/Users/alessandropesare/anaconda3/envs/DL_Project/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [39]:
string_to_vec = vectorizer.transform([string])
lem_string_to_vec = vectorizer_lem.transform([lemmatized_string])

In [40]:
print(cl.predict(string_to_vec))
print(cl_lem.predict(lem_string_to_vec))

[0]
[1]
